# SERVER SEARCHER

A script to find all the sites associated with a IP address' Network ID component (using the DomainBigData database).

Returns a dataframe with all found websites as well as key registration details. 

Also looks to see if website is active.

Accepts either full IP (eg '103.11.212.177') or network ID (eg '103.11.212')

In [4]:
## Libraries
import requests
from bs4 import BeautifulSoup, Tag
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import networkx as nx
import math
import urllib

In [5]:
### get info from DBD page using FIND / FINDALL
def getInfobit(idwords, soup):
    try:
        infobit = soup.find('tr', id=idwords).find_all('td')[1].contents[0]
        infobit = infobit.contents[0] if '<a' in str(infobit) else infobit
    except:
        infobit = None
    return infobit

In [21]:
def serverSearcher(networkID):
    
    if len(networkID.split('.')) ==3:
        ip_A, ip_B, ip_C = networkID.split('.')
    else:
        ip_A, ip_B, ip_C,_ = networkID.split('.')
        
    df = pd.DataFrame(columns=['IP', 'Website', 'Title', 'Date Creation', 'Registrant Organization', 'Registrant Name', 'Registrant Email', 'Status'])

    print(f'{ip_A}.{ip_B}.{ip_C}.X')

    for ip_D in range(0, 256): 
                        
        ip = f'{ip_A}.{ip_B}.{ip_C}.{ip_D}'
            
        print(ip)

        dbd_ip_url = f'https://domainbigdata.com/{ip}'

        page = requests.get(dbd_ip_url)                                
        soup = BeautifulSoup(page.content, 'html.parser')

        websites_on_ip = soup.find(id='MainMaster_divRptDomainsOnSameIP')

        if websites_on_ip != None:

            websites_on_ip = websites_on_ip.find_all("a")
            for x in websites_on_ip:

                url_slug = x.contents[0]
                actual_url = 'http://www.' + url_slug

                next_row = df.shape[0]+1                                     ### add to df
                df.loc[next_row,'IP'] = ip
                df.loc[next_row,'Website'] = url_slug
                    
                
            
                ### Is site live?
                try:
                    page2 = requests.get(actual_url)                                
                    soup2 = BeautifulSoup(page2.content, 'html.parser')
                    df.loc[next_row,'Status'] = 'Live'
                except:
                    df.loc[next_row,'Status'] = 'Not Live'


                ### get site's Registration details

                url_dbd_url = f'https://domainbigdata.com/{url_slug}'

                page3 = requests.get(url_dbd_url)                                
                soup3 = BeautifulSoup(page3.content, 'html.parser')

                df.loc[next_row,'Title'] = getInfobit('trTitle', soup3)
                df.loc[next_row,'Date Creation'] = getInfobit('trDateCreation', soup3)
                df.loc[next_row,'Registrant Organization'] = getInfobit('MainMaster_trRegistrantOrganization', soup3)
                df.loc[next_row,'Registrant Name'] = getInfobit('trRegistrantName', soup3)
                df.loc[next_row,'Registrant Email'] = getInfobit('trRegistrantEmailInRpt', soup3)
                
                print(ip, '\t', actual_url, '\t', getInfobit('trTitle', soup3), '\t', getInfobit('MainMaster_trRegistrantOrganization', soup3), '\t', getInfobit('trRegistrantName', soup3))

    return df 

In [22]:
df = serverSearcher('104.18.27.0')

104.18.27.X
104.18.27.0
104.18.27.1
104.18.27.1 	 http://www.zvmjko.live 	 哪里有十三水游戏下 	 None 	 None
104.18.27.1 	 http://www.ujgekc.live 	 阿里牛牛官网 	 None 	 None
104.18.27.1 	 http://www.mgvpzj.live 	 捕鱼游戏金币 	 None 	 None
104.18.27.1 	 http://www.mnomtf.live 	 nba赛程季后赛 	 None 	 None
104.18.27.1 	 http://www.qbehyo.live 	 欢乐升级微信版 	 None 	 None
104.18.27.1 	 http://www.elouac.live 	 电子老虎机怎么刷流水 	 None 	 None
104.18.27.1 	 http://www.hbexbi.live 	 免费黑客棋牌透牌器 	 None 	 None
104.18.27.2
104.18.27.2 	 http://www.letmeterriket.pro 	 None 	 Protectservice Ltd 	 None
104.18.27.2 	 http://www.rrtucc.live 	 猴子和香蕉的相同点 	 None 	 None
104.18.27.2 	 http://www.tyfoeg.live 	 电子试玩 	 None 	 None
104.18.27.2 	 http://www.pekeeq.live 	 德甲在线直播观看 	 None 	 None
104.18.27.2 	 http://www.bkiyii.live 	 澳门国际注册送彩金 	 None 	 None
104.18.27.2 	 http://www.bzirup.live 	 nba赛程直播腾讯 	 None 	 None
104.18.27.2 	 http://www.dwpoou.live 	 欢乐升级为什么玩不了 	 None 	 None
104.18.27.2 	 http://www.ftnzyw.live 	 免费扎金花游戏 	 None 	 None
104.18.

In [23]:
df

,IP,Website,Title,Date Creation,Registrant Organization,Registrant Name,Registrant Email,Status
1,104.18.27.1,zvmjko.live,哪里有十三水游戏下,2019-08-07,None,None,None,Not Live
2,104.18.27.1,ujgekc.live,阿里牛牛官网,2019-08-07,None,None,None,Not Live
3,104.18.27.1,mgvpzj.live,捕鱼游戏金币,2019-08-07,None,None,None,Not Live
4,104.18.27.1,mnomtf.live,nba赛程季后赛,2019-08-07,None,None,None,Not Live
5,104.18.27.1,qbehyo.live,欢乐升级微信版,2019-08-07,None,None,None,Not Live
...,...,...,...,...,...,...,...,...
1640,104.18.27.250,lfrcrz.live,pk10四码二期必中方法,2019-08-07,None,None,None,Not Live
1641,104.18.27.250,tbsdhd.live,None,2019-08-07,None,None,None,Not Live
1642,104.18.27.250,pfizermedicalinformationgulf.com,None,2018-07-23,Pfizer Inc. (nyhq),Domain Registrations,None,Live
1643,104.18.27.250,gssdes.live,北京快乐和值8走势图,2019-08-07,None,None,None,Not Live
